#### Importing Libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import pickle


#### Loading DataSet

In [2]:
data=pd.read_csv('Churn_Modelling.csv')
data['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [3]:
## drop irrelevant feature
data=data.drop(['Surname','CustomerId','RowNumber'],axis=1)
data['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64

In [4]:
## Encoding Categorical Column
label_encoder=LabelEncoder()
data['Gender']=label_encoder.fit_transform(data['Gender'])

In [5]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
### One hot encoding on geography
from sklearn.preprocessing import OneHotEncoder
one_hot=OneHotEncoder()
geo_encode=one_hot.fit_transform(data[['Geography']])
geo_encode


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
one_hot.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
encoded_data=pd.DataFrame(geo_encode.toarray(),columns=one_hot.get_feature_names_out(['Geography']))
encoded_data

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [9]:
## Combining with orginal data
data=pd.concat([data.drop('Geography',axis=1),encoded_data],axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [10]:
## Train TEst Split

X=data.drop('Exited',axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scaling data
scalar=StandardScaler()
X_train=scalar.fit_transform(X_train)
X_test=scalar.fit_transform(X_test)

In [11]:
## saving encoder
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('onehot.pkl','wb') as file:
    pickle.dump(one_hot,file)

## Saving Scalar
with open('scalar.pkl','wb') as file:
    pickle.dump(scalar,file)



#### **ANN Implementation**
- Sequntial model
- sigmoid - classification
- softmax - multi classification
- I/p neaurons equal to columns
- Dense for hidden neaurons
- optimizer -> Back propagation ->updating weights
- Metrics -> Classification [accuracy] , Regression [mse,mae]
- Training -> Store logs -> Tensorflow ->Visualization

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [13]:
X_train.shape

(8000, 12)

In [14]:
## Building ANN model
model=Sequential([
    ## Input Layer
    ## no of neaurons , activation_function, input_size 
    Dense(16,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(8,activation='relu'),
    Dense(1,activation='sigmoid')
]
)

d:\Gen AI projects\Churn Modelling Project biased results\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353 (1.38 KB)

 Trainable params: 353 (1.38 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()

In [17]:
## Compile forward and backward propagation
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [18]:
## Setup Tensorboard
log_dir='logs/fit'
#  + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [19]:
### Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [20]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7683 - loss: 0.4962 - val_accuracy: 0.8650 - val_loss: 0.3566
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8459 - loss: 0.3607 - val_accuracy: 0.8605 - val_loss: 0.3453
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8629 - loss: 0.3386 - val_accuracy: 0.8595 - val_loss: 0.3395
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8599 - loss: 0.3395 - val_accuracy: 0.8605 - val_loss: 0.3453
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8622 - loss: 0.3422 - val_accuracy: 0.8600 - val_loss: 0.3430
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8637 - loss: 0.3331 - val_accuracy: 0.8645 - val_loss: 0.3371
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8639 - loss: 0.3400 - val_accuracy: 0.8590 - val_loss: 0.3430
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8708 - loss: 0.3205 - val_accu

In [21]:
model.save('mdoel.h5')

In [22]:
## Load Tensornoard Extension
%load_ext tensorboard

In [23]:
%tensorboard --logdir logs/fit

ERROR: Failed to launch TensorBoard (exited with 1).